In [1]:
!pip install keras-tuner
!pip install tensorflow-addons

In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display

from IPython.display import clear_output

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import tensorflow_addons as tfa
import keras_tuner as kt

In [4]:
# mount drive to save generated files
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)
#root_dir = '/content/drive/MyDrive/'
#base_dir = root_dir + 'Music Genre/mfcc/small/'

In [5]:
base_dir = '/kaggle/input/music-genre/' # Kaggle path

In [6]:
train_dir = base_dir + 'training/'
val_dir = base_dir + 'validation/'
test_dir = base_dir + 'test/'

In [7]:
from PIL import Image
from matplotlib import cm

mfcc_data = Image.open(train_dir + 'folk/000_000140_s0.tif')

imarray = np.array(mfcc_data) 
mfcc_shape = imarray.shape
print( mfcc_shape )

ig, ax = plt.subplots()
mfcc_data= np.swapaxes(mfcc_data, 0 ,1)
cax = ax.imshow(mfcc_data, interpolation='nearest', cmap=cm.coolwarm, origin='lower', aspect='auto')
ax.set_title('MFCC')
#Showing mfcc_data
plt.show()

In [8]:
train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

batch_size = 128
epochs = 3

num_classes = 8
input_shape = (mfcc_shape[0], mfcc_shape[1], 1)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(input_shape[0], input_shape[1]),
        color_mode= "grayscale",
        batch_size=batch_size,
        class_mode='sparse')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(input_shape[0], input_shape[1]),
        color_mode= "grayscale",
        batch_size=batch_size,
        class_mode='sparse')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(input_shape[0], input_shape[1]),
        color_mode= "grayscale",
        batch_size=batch_size,
        class_mode='sparse')

In [9]:
precision_function = tf.keras.metrics.Precision()
recall_function = tf.keras.metrics.Recall()
auc_function = tf.keras.metrics.AUC()
f1_function = tfa.metrics.F1Score(num_classes=num_classes, average='micro')

In [10]:
def model_builder_dense(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=input_shape))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(num_classes))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy", f1_function])

  return model

In [11]:
def model_builder_cnn_1(hp):
    inputs = keras.Input(shape=input_shape)

    x = inputs
    # Number of layers of the CNN is also a hyperparameter.
    for i in range(hp.Int("cnn_layers", 1, 3)):
        x = layers.Conv2D(
            hp.Int(f"filters_{i}", 4, 128, step=4),
            kernel_size=(3, 3),
            activation="relu",
        )(x)
        x = layers.MaxPooling2D(pool_size=(1, 2))(x)
    x = layers.Flatten()(x)

    # A hyperparamter for whether to use dropout layer.
    if hp.Boolean("dropout"):
        x = layers.Dropout(hp.Choice('dropout_value', values=[0.1, 0.25, 0.5]))(x)

    # The last layer contains the output units,
    # which is the same as the number of classes.
    outputs = layers.Dense(units=num_classes, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)

    # Compile the model.
    model.compile(
        loss="sparse_categorical_crossentropy", 
        metrics=["accuracy", f1_function], 
        optimizer="adam",
    )
    return model

In [12]:
print( input_shape )

model = keras.Sequential(
    [
     keras.Input(shape=(input_shape)),
     layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
     layers.MaxPooling2D(pool_size=(1, 2)),
     layers.Conv2D(16, kernel_size=(2, 2), activation="relu"),
     layers.MaxPooling2D(pool_size=(1, 2)),
     layers.Conv2D(4, kernel_size=(2, 2), activation="relu"),
     layers.MaxPooling2D(pool_size=(1, 2)),
     layers.Flatten(), #flat the data
     layers.Dropout(0.1),
     layers.Dense(num_classes, activation="softmax")
    ]
)

model.summary()

In [13]:
def model_builder(hp):

    model = keras.Sequential()
    model.add( keras.Input(shape=input_shape) )

    model.add( 
            layers.Conv2D(
            filters=hp.Int("conv_1_filter", 16, 64, step=16),
            kernel_size=(3, 3), activation="relu")
        )
    model.add( layers.MaxPooling2D(pool_size=(1, 2)) )

    model.add( 
            layers.Conv2D(
            filters=hp.Int("conv_2_filter", 4, 32, step=4),
            kernel_size=(3, 3), activation="relu")
        )
    model.add( layers.MaxPooling2D(pool_size=(1, 2)) )

    model.add( 
            layers.Conv2D(
            filters=hp.Int("conv_3_filter", 4, 16, step=4),
            kernel_size=(3, 3), activation="relu")
        )
    model.add( layers.MaxPooling2D(pool_size=(1, 2)) )

    model.add( layers.Flatten() )

    model.add( 
        layers.Dropout(hp.Choice('dropout_value', values=[0.1, 0.25, 0.5])) 
        )
    
    model.add( layers.Dense(units=num_classes, activation="softmax") )

    model.compile( 
        loss="sparse_categorical_crossentropy", 
        metrics=["accuracy", f1_function],
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])) )
    
    return model

In [14]:
def model_builder_cnn(hp):
    inputs = keras.Input(shape=input_shape)

    x = inputs
    filt_size = hp.Int("filters_size", 32, 128, step=32, default=64)

    # Number of layers of the CNN is also a hyperparameter.
    if filt_size > 32:
      num_layers = hp.Int("cnn_layers", 1, 3)
    else:
      num_layers = hp.Int("cnn_layers", 1, 2)

    # First layer
    x = layers.Conv2D(
          filt_size,
          kernel_size=(3, 3),
          activation="relu",
      )(x)
    x = layers.MaxPooling2D(pool_size=(1, 2))(x)

    for i in range( num_layers - 1  ):
      resize_ratio = hp.Choice(f'ratio_{i+1}_layers', values=[1, 2, 4] , default=2)
      x = layers.Conv2D(
          int(filt_size / resize_ratio),
          kernel_size=(2, 2),
          activation="relu",
      )(x)
      x = layers.MaxPooling2D(pool_size=(1, 2))(x)

    # Flat
    x = layers.Flatten()(x) 
    # Dropout
    x = layers.Dropout(hp.Choice('dropout_value', values=[0.75, 0.5, 0.25]))(x)

    outputs = layers.Dense(units=num_classes, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)

    # Compile the model.
    model.compile(
        loss="sparse_categorical_crossentropy", 
        metrics=["accuracy", f1_function], 
        optimizer="adam",
    )
    return model

In [15]:
tuner = kt.Hyperband(model_builder_cnn,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3)

In [16]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [17]:
tuner.search(train_generator, epochs=50, validation_data=validation_generator, callbacks=[stop_early])
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [18]:
# Build the model with the optimal hyperparameters 
model = tuner.hypermodel.build(best_hps)

model.save('cnn_model_after_hps') # to save the model
#model = keras.models.load_model('cnn_model_after_hps') # to load the model

In [ ]:
model.summary()

In [19]:
# Train it on the data for 50 epochs
history = model.fit(train_generator, epochs=50, validation_data=validation_generator)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [20]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
network_history = hypermodel.fit(train_generator, epochs=best_epoch, validation_data=validation_generator)

In [21]:
hypermodel.summary()

In [ ]:
hypermodel.save('cnn_model') # to save the model
#hypermodel = keras.models.load_model('cnn_model') # to load the model

In [23]:
eval_result = hypermodel.evaluate(test_generator)
print("[test loss, test accuracy]:", eval_result)

In [24]:
from matplotlib import pyplot as plt
x_plot = list(range(1,best_epoch+1))

def plot_history(network_history):
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(x_plot, network_history.history['loss'])
    plt.plot(x_plot, network_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(x_plot, network_history.history['accuracy'])
    plt.plot(x_plot, network_history.history['val_accuracy'])
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()

In [25]:
plot_history(network_history)

In [26]:
test_pred = model.predict(test_generator)

In [27]:
y_pred = []
for pred in test_pred:
  y_pred.append( np.argmax(pred) )

In [28]:
from sklearn.metrics import f1_score

y_true = test_generator.labels

f1_test = f1_score(y_true, y_pred, average='macro')
print('Average f1_score: {} \n' .format(f1_test) )

print('F1-SCORE FOR EACH CLASS')
print('-----------------------')
av_f1_score = f1_score(y_true, y_pred, average=None)
for i in range(len(av_f1_score)):
  print('{} : {} '.format( i, av_f1_score[i]))

In [29]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay( confusion_matrix=cm )

disp.plot()
frame1 = plt.gca()
frame1.axes.get_xaxis().set_visible(False)
plt.show()

In [30]:
!zip -r file.zip '/kaggle/working/cnn_model'

In [31]:
hypermodel_2 = keras.models.load_model('cnn_model') 
hypermodel_2.summary()